# Running Plato in a Terminal with Google Colab and Visual Studio Code

It is strongly recommended and more convenient to run Plato in a terminal, preferably in Visual Studio Code. This notebook contains step-by-step instructions on how this can be done when Google Colab is used. 



## 1. Changing your Runtime Type

If you have subscribed to Google Colab Pro or Pro+, you may select *Runtime > Change runtime type* and then select *High-RAM* for *Runtime shape* and *Background execution* (only available to Colab Pro+).

Now you may use the following code to check the GPU type allocated to you in the current session:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

You can also use the following code to verify the amount of physical memory available in your active session:

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## 2. Mounting your Google Drive

Since Google Colab removes all the files that you may have downloaded or created when you terminal a session, the best option is to use GitHub to store your code, and Google Drive to store your downloaded datasets and anything else needed by your Plato training sessions that you would normally store on your local filesystem.

You can use the code below to mount your Google Drive, which may contain your downloaded datasets. When you run the code below, you will need to click a link and follow a process that takes a few seconds. When the process is complete, all of your Google Drive files will be available at `/content/drive/MyDrive` on your Colab instance.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Connecting to the Remote Server

You are now ready to remotely connect to the Google Colab host in Visual Studio Code. If this is the first time you do it, please download and install [Cloudflared](https://developers.cloudflare.com/argo-tunnel/getting-started/installation). In macOS, for example, the command to install it will be `brew install cloudflare/cloudflare/cloudflared`.

After installing `cloudflared`, you should add the following entry into `~/.ssh/config`:

```
Host *.trycloudflare.com
	HostName %h
	User root
	Port 22
	ProxyCommand PUT_THE_ABSOLUTE_CLOUDFLARED_PATH_HERE access ssh --hostname %h
```

Where the absolute path to `cloudflared` on macOS, for example, is `/opt/homebrew/bin/cloudflared`. You do not need to perform these steps again later on —— you only need to do them once.

In [ ]:
!pip install colab_ssh --upgrade

from getpass import getpass
password = getpass('Please enter the login password for accessing the remote server: ')

from colab_ssh import launch_ssh_cloudflared, init_git_cloudflared
launch_ssh_cloudflared(password=password)

repository_url="https://github.com/TL-System/plato"
init_git_cloudflared(repository_url)

Now click the "Open plato" button, and enter the login password you have just provided. You will now be connected to the remote Colab server in Visual Studio Code. You can open a terminal in Visual Studio Code to access the remote server at this time.

## 4. Setting Up your Python Environment

Before running Plato, make sure that uv is installed, and you can run Plato using `uv run`.
